In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['your_database_name']
collection = db['your_collection_name']


In [5]:
import pprint
pprint.pprint(list(collection.index_information()))


[]


In [3]:
collection.drop_index('index_name')


In [4]:
class MongoDBManager:
    def __init__(self, uri="mongodb://localhost:27017/"):
        self.client = MongoClient(uri)
        self.db = self.client['article_db']
        self.article_collection = self.db['articles']

    def drop_all_indexes(self):
        """Drop all indexes on the article collection except for the default _id index."""
        self.article_collection.drop_indexes()

    def recreate_indexes(self):
        """Create necessary indexes according to the new schema."""
        self.article_collection.create_index("metadata.url", unique=True)
        self.article_collection.create_index("metadata.published_date")
        self.article_collection.create_index("metadata.keywords")


In [2]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['article_db']
collection = db['articles']

# Option 1: Delete All Documents in the Collection
def delete_all_documents():
    """Deletes all documents in the 'articles' collection."""
    result = collection.delete_many({})
    print(f"Deleted {result.deleted_count} documents from the collection.")

# Option 2: Drop the Entire Collection
def drop_collection():
    """Drops the entire 'articles' collection."""
    collection.drop()
    print("Collection 'articles' has been dropped.")

# Run either function as needed
# delete_all_documents()  # Deletes all documents
drop_collection()     # Drops the collection entirely


Collection 'articles' has been dropped.


In [17]:
from pymongo import MongoClient
import pprint

client = MongoClient("mongodb://localhost:27017/")
db = client['article_db']
collection = db['articles']

def aggregate_nested_schema(collection_name, sub_document):
    collection = db[collection_name]
    pipeline = [
        {"$match": {sub_document: {"$exists": True}}},  # Ensure the sub-document exists
        {"$project": {sub_document: {"$objectToArray": f"${sub_document}"}}},
        {"$unwind": f"${sub_document}"},
        {"$group": {"_id": None, "allKeys": {"$addToSet": f"${sub_document}.k"}}}
    ]
    results = collection.aggregate(pipeline)
    for result in results:
        print(f"All keys in {sub_document} of collection {collection_name}:")
        pprint.pprint(result['allKeys'])

# Example usage for 'metadata' and 'summary' sub-documents
aggregate_nested_schema('articles', 'metadata')
aggregate_nested_schema('articles', 'summary')


All keys in metadata of collection articles:
['url',
 'title',
 'content',
 'authors',
 'published_date',
 'processing_date',
 'keywords']
All keys in summary of collection articles:
['model_used', 'text']


In [1]:
from pymongo import MongoClient
import pprint

client = MongoClient("mongodb://localhost:27017/")
db = client['article_db']
users_collection = db['users']

def aggregate_schema(collection):
    pipeline = [
        {"$project": {"arrayOfKeyValue": {"$objectToArray": "$$ROOT"}}},
        {"$unwind": "$arrayOfKeyValue"},
        {"$group": {"_id": None, "allKeys": {"$addToSet": "$arrayOfKeyValue.k"}}}
    ]
    results = collection.aggregate(pipeline)
    for result in results:
        print(f"Keys found in {collection.name}:")
        pprint.pprint(result['allKeys'])

def aggregate_nested_schema(collection_name, sub_document):
    collection = db[collection_name]
    pipeline = [
        {"$match": {sub_document: {"$exists": True}}},  # Ensure the sub-document exists
        {"$project": {sub_document: {"$objectToArray": f"${sub_document}"}}},
        {"$unwind": f"${sub_document}"},
        {"$group": {"_id": None, "allKeys": {"$addToSet": f"${sub_document}.k"}}}
    ]
    results = collection.aggregate(pipeline)
    for result in results:
        print(f"All keys in {sub_document} of collection {collection_name}:")
        pprint.pprint(result['allKeys'])

# Get overall schema of the users collection
aggregate_schema(users_collection)

# If you know there are nested documents, such as a nested 'articles' or 'profiles', analyze them too
aggregate_nested_schema('users', 'articles')
aggregate_nested_schema('users', 'profiles')  # Change 'profiles' to the actual sub-document name if different


Keys found in users:
['email', 'articles', '_id']
All keys in articles of collection users:
['28102024']


In [8]:
import nltk
print(nltk.data.path)

['/Users/aydin/nltk_data', '/Users/aydin/Documents/aldi-projects/genai-newsflash/.venv/nltk_data', '/Users/aydin/Documents/aldi-projects/genai-newsflash/.venv/share/nltk_data', '/Users/aydin/Documents/aldi-projects/genai-newsflash/.venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [2]:
from pymongo import MongoClient
import pprint

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['article_db']
collection = db['articles']  # Replace 'articles' with your collection name

# Check if the collection has any documents and print them
def check_collection():
    # Check if any documents exist in the collection
    if collection.count_documents({}) > 0:
        print("Documents found in the collection:")
        for document in collection.find():
            pprint.pprint(document)  # Pretty-print each document
    else:
        print("No documents found in the collection.")

# Run the check
check_collection()


Documents found in the collection:
{'_id': ObjectId('671f5763fc1603d19ae9352b'),
 'metadata': {'authors': ['Ben Tossell'],
              'content': 'Teachers today have more tools at their disposal '
                         'than ever before. Many AI apps also offer free '
                         'features that can make daily tasks easier. But how '
                         'can you navigate this noisy marketplace and find the '
                         'right tools for your teaching needs?\n'
                         '\n'
                         'This post covers the best free AI tools available to '
                         "teachers. We'll look at the key features of each "
                         'tool and how they can be useful in the classroom. '
                         'Side note: many of these tools have optional paid '
                         'plans, but you can get started with the free version '
                         'and pay to upgrade if/when you need.\n'
        

In [3]:
from pymongo import MongoClient
import pprint

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['article_db']
collection = db['users']  # Replace 'articles' with your collection name

# Check if the collection has any documents and print them
def check_collection():
    # Check if any documents exist in the collection
    if collection.count_documents({}) > 0:
        print("Documents found in the collection:")
        for document in collection.find():
            pprint.pprint(document)  # Pretty-print each document
    else:
        print("No documents found in the collection.")

# Run the check
check_collection()

Documents found in the collection:
{'_id': ObjectId('671f5763d75b84aa4186c4b0'),
 'articles': {'28102024': ['671f5763fc1603d19ae9352b',
                           '671f576dfc1603d19ae9352c',
                           '671f577afc1603d19ae9352d',
                           '671feba01820fa5d57dbcc97',
                           '671febaa1820fa5d57dbcc98',
                           '671febb81820fa5d57dbcc99',
                           '671febc61820fa5d57dbcc9a',
                           '671febd01820fa5d57dbcc9b',
                           '671febd71820fa5d57dbcc9c',
                           '671febe11820fa5d57dbcc9d',
                           '671febe81820fa5d57dbcc9e',
                           '671febf11820fa5d57dbcc9f',
                           '671febfc1820fa5d57dbcca0',
                           '671fec041820fa5d57dbcca1',
                           '671fec0e1820fa5d57dbcca2',
                           '671fec211820fa5d57dbcca3',
                           '671fec29182

In [4]:
# ... existing code ...

# Check if the collection has any documents and print them
def check_collection():
    # Check if any documents exist in the collection
    if collection.count_documents({}) > 0:
        total_articles = 0  # Counter for total articles
        print(f"{collection.count_documents({})} Documents found in the collection:")
        for document in collection.find():
            pprint.pprint(document)  # Pretty-print each document
            # Count articles in the current document
            if 'users' in document:
                total_articles += len(document['articles'])

        print(f"Total articles in the collection: {total_articles}")  # Print total articles
    else:
        print("No documents found in the collection.")

# Run the check
check_collection()

3 Documents found in the collection:
{'_id': ObjectId('671f5763d75b84aa4186c4b0'),
 'articles': {'28102024': ['671f5763fc1603d19ae9352b',
                           '671f576dfc1603d19ae9352c',
                           '671f577afc1603d19ae9352d',
                           '671feba01820fa5d57dbcc97',
                           '671febaa1820fa5d57dbcc98',
                           '671febb81820fa5d57dbcc99',
                           '671febc61820fa5d57dbcc9a',
                           '671febd01820fa5d57dbcc9b',
                           '671febd71820fa5d57dbcc9c',
                           '671febe11820fa5d57dbcc9d',
                           '671febe81820fa5d57dbcc9e',
                           '671febf11820fa5d57dbcc9f',
                           '671febfc1820fa5d57dbcca0',
                           '671fec041820fa5d57dbcca1',
                           '671fec0e1820fa5d57dbcca2',
                           '671fec211820fa5d57dbcca3',
                           '671fec291

In [11]:
import logging
import json
import time
import yaml
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import RatelimitException
from datetime import datetime

In [12]:
# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Create a logger for this module
logger = logging.getLogger(__name__)

In [40]:
import logging
import time
from typing import List, Dict
from pydantic import BaseModel, Field
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import RatelimitException
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
# from helpers import load_config

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Add file handler for logging
file_handler = logging.FileHandler('app.log')
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)

def load_config():
    """Load configuration from the config.yml file."""
    with open('./config.yml', 'r') as file:
        return yaml.safe_load(file)


class SearchResult(BaseModel):
    """Schema for a single search result evaluation"""
    url: str = Field(description="The URL of the search result")
    score: int = Field(description="Relevance score from 0-10")

class SearchEvaluations(BaseModel):
    """Schema for all search result evaluations"""
    results: List[SearchResult] = Field(description="List of evaluated search results")

class SearchModule:
    def __init__(self):
        config = load_config()
        self.model_name = config.get("openai_model_name")
        self.api_key = config.get("OPENAI_API_KEY")
        self._initialize_llm()

    def _initialize_llm(self):
        """Initialize LangChain components"""
        if self.api_key:
            self.llm = ChatOpenAI(
                api_key=self.api_key,
                model_name=self.model_name,
                temperature=0.1
            )
            self.parser = PydanticOutputParser(pydantic_object=SearchEvaluations)
        else:
            logger.error("OpenAI API key not found")
            self.llm = None
            self.parser = None

    def _create_evaluation_chain(self) -> LLMChain:
        """Create the evaluation chain with prompt and output parser"""
        template = """You are evaluating search results for relevance to these keywords: {keywords}

For each result, rate its relevance from 0-10 based on:
1. Direct mention or discussion of the keywords
2. Recency and relevance of the content
3. Authority and reliability of the source

Consider the context and provide accurate numerical scores.

Search Results:
{search_results}

{format_instructions}

Important: Focus on results with clear relevance to the keywords and current context."""

        prompt = ChatPromptTemplate.from_template(
            template=template,
            partial_variables={
                "format_instructions": self.parser.get_format_instructions()
            }
        )

        return LLMChain(
            llm=self.llm,
            prompt=prompt
        )

    def search_duckduckgo(self, keywords: str, timelimit: str = 'w', max_results: int = 20, max_retries: int = 3) -> List[Dict]:
        """Perform a DuckDuckGo search for the specified keywords."""
        logger.info(f"Searching DuckDuckGo for query: {keywords}")
        with DDGS() as ddgs:
            retry_count = 0
            while retry_count < max_retries:
                try:
                    results = []
                    for r in ddgs.text(keywords, timelimit=timelimit, max_results=max_results):
                        logger.debug(f"Scraped result: {r}")
                        results.append(r)
                        time.sleep(3)  # Rate limit handling
                    if results:
                        logger.info(f"Successfully retrieved {len(results)} results for query: {keywords}")
                        return results
                    else:
                        logger.warning(f"No results found for query: {keywords}")
                        return []
                except RatelimitException:
                    retry_count += 1
                    logger.warning(f"RatelimitException encountered. Retrying in {2 ** retry_count} seconds.")
                    time.sleep(2 ** retry_count)
                except Exception as e:
                    logger.error(f"An error occurred during search: {e}")
                    break
        logger.error(f"Failed to retrieve results for query: {keywords} after {max_retries} retries")
        return []

    def _format_search_results(self, search_results: List[Dict]) -> str:
        """Format search results for the prompt"""
        formatted_results = []
        for i, item in enumerate(search_results, 1):
            result = (
                f"Result {i}:\n"
                f"Title: {item['title']}\n"
                f"URL: {item['href']}\n"
                f"Description: {item.get('body', '')[:300]}\n"
            )
            formatted_results.append(result)
        return "\n\n".join(formatted_results)

    def filter_relevant_links(self, search_results: List[Dict], keywords: List[str], min_score: int = 6) -> List[str]:
        """
        Filter search results for relevance to provided keywords.

        Args:
            search_results: List of dictionaries containing search results
            keywords: List of keywords to evaluate relevance against
            min_score: Minimum relevance score (0-10) for inclusion

        Returns:
            List of relevant URLs that meet the minimum score threshold
        """
        if not self.api_key or not self.llm:
            logger.error("OpenAI API key not found or LLM not initialized")
            return []

        try:
            # Create and run the evaluation chain
            chain = self._create_evaluation_chain()

            response = chain.run({
                "keywords": ", ".join(keywords),
                "search_results": self._format_search_results(search_results)
            })

            # Parse response and extract relevant links
            evaluations = self.parser.parse(response)
            relevant_links = []

            for result in evaluations.results:
                if result.score >= min_score:
                    relevant_links.append(result.url)
                    logger.info(f"Relevant link added: {result.url} with score {result.score}")
                else:
                    logger.debug(f"Link excluded: {result.url} with score {result.score}")

            return relevant_links

        except Exception as e:
            logger.error(f"Link filtering failed: {str(e)}", exc_info=True)
            return []

# Example usage
if __name__ == "__main__":
    search_module = SearchModule()

    # Example search and filter
    keywords = ["cloud computing", "digital transformation"]
    search_results = search_module.search_duckduckgo(" ".join(keywords))

    if search_results:
        relevant_links = search_module.filter_relevant_links(
            search_results=search_results,
            keywords=keywords
        )
        print(f"Found {len(relevant_links)} relevant links")

ModuleNotFoundError: No module named 'helpers'

In [21]:
search_module = SearchModule()

# Example search and filter
keywords = ["cloud computing", "digital transformation"]
search_results = search_module.search_duckduckgo(" ".join(keywords))

if search_results:
    relevant_links = search_module.filter_relevant_links(
        search_results=search_results,
        keywords=keywords
    )
    print(f"Found {len(relevant_links)} relevant links")

2024-10-30 22:10:00,858 - __main__ - INFO - Searching DuckDuckGo for query: Schwarz IT
2024-10-30 22:10:00,995 - primp - INFO - response: https://duckduckgo.com/?q=Schwarz+IT 200 17678
2024-10-30 22:11:01,833 - __main__ - INFO - Successfully retrieved 20 results for query: Schwarz IT


[{'title': 'Digital sovereignty thanks to the German cloud: Schwarz Digits and SAP ...',
  'href': 'https://schwarz-digits.de/en/presse/archive/2024/digital-sovereignty-thanks-to-the-german-cloud-schwarz-digits-and-sap-will-offer-rise-with-sap-on-stackit-in-future',
  'body': "Companies of Schwarz Group will also be able to move their entire ERP landscape to the STACKIT Cloud via RISE with SAP. Back in 2023, SAP and Schwarz Group companies announced their partnership at Sapphire in Orlando to protect both SAP's IT infrastructure and its customer-specific cloud services and RISE with XM Cyber."},
 {'title': 'Schwarz IT Interview Questions (2024) - Glassdoor',
  'href': 'https://www.glassdoor.com/Interview/Schwarz-IT-Interview-Questions-E3046992.htm',
  'body': 'Vorstellung der Gespächsteilnehmer sowie der Schwarz Gruppe, durchführen durch den CV danach stellenbezogene Fragen und Fachfragen. Am Ende konnte ich dann Fragen stellen. Zum Schluss wurde dann noch das weitere Vorgehen erläuter

In [36]:
def load_config():
    """Load configuration from the config.yml file."""
    with open('./config.yml', 'r') as file:
        return yaml.safe_load(file)


In [37]:
import logging
import json
import time
import yaml
import os
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import RatelimitException
from datetime import datetime
from langchain_openai import ChatOpenAI


# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Optionally, add a file handler for logging
file_handler = logging.FileHandler('app.log')
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)

# Load configuration


class SearchModule:
    def __init__(self):
        config = load_config()
        self.model_name = config.get("openai_model_name")
        self.api_key = config.get("OPENAI_API_KEY")

    def search_duckduckgo(self, keywords, timelimit='w', max_results=20, max_retries=3):
        """Perform a DuckDuckGo search for the specified keywords."""
        logger.info(f"Searching DuckDuckGo for query: {keywords}")
        with DDGS() as ddgs:
            retry_count = 0
            while retry_count < max_retries:
                try:
                    results = []
                    for r in ddgs.text(keywords, timelimit=timelimit, max_results=max_results):
                        logger.debug(f"Scraped result: {r}")
                        results.append(r)
                        time.sleep(3)  # Rate limit handling
                    if results:
                        logger.info(f"Successfully retrieved {len(results)} results for query: {keywords}")
                        return results
                    else:
                        logger.warning(f"No results found for query: {keywords}")
                        return []
                except RatelimitException:
                    retry_count += 1
                    logger.warning(f"RatelimitException encountered. Retrying in {2 ** retry_count} seconds.")
                    time.sleep(2 ** retry_count)
                except Exception as e:
                    logger.error(f"An error occurred during search: {e}")
                    break
        logger.error(f"Failed to retrieve results for query: {keywords} after {max_retries} retries")
        return []

    def filter_relevant_links(self, search_results: list, keywords: list) -> list:
        """Filter search results for relevance to provided keywords, keeping results with a score of 6 or higher."""
        if not self.api_key:
            logger.error("OpenAI API key not found")
            return []

        try:
            llm = ChatOpenAI(api_key=self.api_key, model=self.model_name)

            # Construct prompt with all items in a single call
            prompt = (
                f"For each search result below, evaluate its relevance to the following keywords: {', '.join(keywords)}. "
                "Assign a score from 0 (not relevant) to 10 (highly relevant) for each item, and provide only the links "
                "that score 6 or above."
            )

            items_text = "\n\n".join(
                f"{i+1}. Title: {item['title']}\nLink: {item['href']}\nDescription: {item['body'][:500]}"
                for i, item in enumerate(search_results)
            )

            messages = [
                {"role": "system", "content": prompt},
                {"role": "user", "content": items_text}
            ]

            response = llm.invoke(messages, temperature=0.1, max_tokens=1000)
            result_text = response.content.strip()

            relevant_links = []
            for i, item in enumerate(search_results):
                if f"{i+1}." in result_text and "score: 6" in result_text:
                    relevant_links.append(item['href'])
                    logger.info(f"Relevant link added: {item['href']} with score 6 or above")

            return relevant_links

        except Exception as e:
            logger.error(f"Link filtering failed: {str(e)}", exc_info=True)
            return []

In [39]:
result_text

NameError: name 'result_text' is not defined

In [38]:
search_module = SearchModule()
# Define keywords for the search
keywords = "digital sovereignty SAP cloud"
search_results = search_module.search_duckduckgo(keywords, timelimit='w', max_results=5)

# Define keywords for filtering relevance
filter_keywords = ["digital sovereignty", "SAP", "cloud"]

# Filter relevant links
relevant_links = search_module.filter_relevant_links(search_results, filter_keywords)

print("Relevant Links:", relevant_links)


2024-10-30 23:04:21,505 - __main__ - INFO - Searching DuckDuckGo for query: digital sovereignty SAP cloud


2024-10-30 23:04:21,647 - primp - INFO - response: https://duckduckgo.com/?q=digital+sovereignty+SAP+cloud 200 18037
2024-10-30 23:04:37,370 - __main__ - INFO - Successfully retrieved 5 results for query: digital sovereignty SAP cloud
2024-10-30 23:04:37,380 - __main__ - ERROR - Link filtering failed: 1 validation error for ChatOpenAI
model
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
Traceback (most recent call last):
  File "/var/folders/mj/z4l2c3ps4dgfzzg6_jhxs8yr0000gn/T/ipykernel_9216/2504772937.py", line 68, in filter_relevant_links
    llm = ChatOpenAI(api_key=self.api_key, model=self.model_name)
  File "/Users/aydin/Documents/aldi-projects/genai-newsflash/.venv/lib/python3.10/site-packages/langchain_core/load/serializable.py", line 111, in __init__
    super().__init__(*args, **kwargs)
  File "/Users/aydin/Documents/aldi-projects/genai-newsflash/.venv/li

Relevant Links: []
